In [0]:
RAW_PATH = "/Volumes/ecommerce/ecommerce/data"
GCP_PROJECT = "regal-elf-481622-u5"
BQ_DATASET = "ecommerce"
TEMP_GCS_BUCKET = "ecommerce-data1"

GCP_SECRET_SCOPE = "gcp-secrets"
GCP_SECRET_KEY = "gcp-sa-key"

In [0]:
from pyspark.sql.functions import (
    col, count, sum, avg, max as fmax, min as fmin, date_trunc, to_date, datediff, current_date, 
    expr, row_number
)
from pyspark.sql.window import Window

df = spark.table("ecommerce.silver.transaction_enriched")

if "net_sales" not in df.columns:
    df = df.withColumn("net_sales", col("total_amount"))

In [0]:
gold_daily_store_cat = spark.sql("""
                                 select transaction_date, store_name, city, category,sum(quantity)as total_units, sum(coalesce(net_sales,total_amount)) as net_sales,
                                 sum(coalesce(discount_amount,0))as discount_given, count(*) as transactions,
                                 avg(coalesce(net_sales,total_amount)) as avg_order_value
                                 from ecommerce.silver.transaction_enriched
                                 group by transaction_date,store_name,city,category
                                 order by transaction_date desc, net_sales desc
                                 """)

gold_daily_store_cat.write.mode("overwrite").option("overwriteSchema", "true").partitionBy("transaction_date").format("delta").saveAsTable("ecommerce.gold.daily_store_category")

In [0]:
gold_top_customers = spark.sql("""
                               select customer_id, customer_name,email, customer_country,
                               sum(coalesce(net_sales,total_amount)) as lifetime_net_spend, count(*) as transaction_count, avg(coalesce(net_sales,total_amount)) as avg_order_value,
                               max(transaction_date) as last_purchase_date
                               from ecommerce.silver.transaction_enriched
                               group by customer_id, customer_name, email, customer_country
                               order by lifetime_net_spend desc
                               """)
gold_top_customers.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("ecommerce.gold.top_customers")

In [0]:
gold_promotion_impact = spark.sql("""
                                  select promotion_method,count(*) as transactions,sum(coalesce(net_sales, total_amount)) as net_sales,sum(coalesce(discount_amount, 0)) as discount_given,avg(discount_percent_applied) as avg_discount_percent,
                                  avg(coalesce(net_sales, total_amount)) as avg_order_value
                                  from ecommerce.silver.transaction_enriched
                                  group by promotion_method
                                  order by net_sales desc
                                  """)
gold_promotion_impact.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("ecommerce.gold.promo_impact")

In [0]:
gold_product_impact = spark.sql("""
                                select product_id,product_name,category,supplier_name, sum(quantity) as units_sold,sum(coalesce(net_sales, total_amount)) as net_sales, count(*) as transactions,avg(coalesce(net_sales, total_amount)) as avg_order_value,max(avg_rating) as avg_rating,max(review_count) as review_count
                                from ecommerce.silver.transaction_enriched
                                group by product_id,product_name, category, supplier_name
                                order by net_sales desc
                                """)
gold_product_impact.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("ecommerce.gold.product_impact")

In [0]:
rfm = spark.sql("""
                with base as(
                select customer_id,customer_name,email,max(transaction_date) as last_purchase_date,count(*)as frequency,sum(coalesce(net_sales, total_amount)) as monetary,datediff(current_date(), 
                max(transaction_date)) as recency_days
                from ecommerce.silver.transaction_enriched
                group by customer_id, customer_name, email
                ),
    scored as (
        select *,
        ntile(5) over (order by recency_days asc) as r_score,
        ntile(5) over (order by frequency desc) as f_score,
        ntile(5) over (order by monetary desc) as m_score
        from base
)
select *, 
  (r_score + f_score + m_score) as rfm_score
  from scored;
""")

rfm.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("ecommerce.gold.rfm")


In [0]:
gold_suspects = spark.sql("""
                          with ordered as (
                          select transaction_id,customer_id,transaction_ts,store_id,store_name,city,
                          coalesce(net_sales, total_amount) as net_sales,
                          lag(transaction_ts) over(partition by customer_id order by transaction_ts) as prev_ts,
                          lag(store_id) over (partition by customer_id order by transaction_ts) as prev_store
                          FROM ecommerce.silver.transaction_enriched
                        ),
        diffed AS (
                select *,
                (unix_timestamp(transaction_ts) - unix_timestamp(prev_ts)) / 60.0 as minutes_diff
                FROM ordered
                )
SELECT
  transaction_id,
  customer_id,
  transaction_ts,
  store_id,
  store_name,
  city,
  net_sales,
  prev_ts,
  prev_store,
  minutes_diff
FROM diffed
WHERE
  prev_ts IS NOT NULL
  AND minutes_diff <= 5
  AND store_id <> prev_store;
""")

gold_suspects.write.mode("overwrite").option("overwriteSchema", "true").format("delta").saveAsTable("ecommerce.gold.suspects")



In [0]:
for t in [
    "ecommerce.gold.promo_impact",
    "ecommerce.gold.product_impact",
    "ecommerce.gold.rfm",
    "ecommerce.gold.suspects",
    "ecommerce.gold.top_customers",
    "ecommerce.gold.daily_store_category"
]:
    c = spark.read.table(t).count()
    print(t, "=>", c)


In [0]:
%sql 

SELECT
  SUM(net_sales) AS gold_net_sales
FROM ecommerce.gold.daily_store_category;


In [0]:
%sql

SELECT
  SUM(COALESCE(net_sales, total_amount)) AS silver_net_sales
FROM ecommerce.silver.transaction_enriched;
